# Notebook 01 | Transformações Silver

Objetivos:

- Carregar as tabelas Bronze registradas em `/FileStore/bronze`  
- Realizar joins entre Sales, Products, Stores e Exchange Rates  
- Converter todos os valores para USD  
- Tratar inconsistências, nulos e tipos de dados  
- Gravar a camada Silver como Delta Table pronta para modelagem  

## 1. Configuração do Ambiente

Inicializar Spark, importar funções e definir banco e caminho da camada Silver.


In [0]:
# Iniciar Spark e importar funções necessárias
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_date, regexp_replace, col
from pyspark.sql.types import DoubleType, IntegerType

spark = SparkSession.builder.getOrCreate()

# Definir nomes de banco e caminho no DBFS
bronze_db   = "bronze"
silver_db   = "silver"
silver_path = "/FileStore/silver/sales_silver"

## 2. Carregamento das Tabelas Bronze

Ler as Delta Tables registradas no catálogo `bronze`.



In [0]:
# Carregar dados Bronze via catálogo
sales    = spark.table(f"{bronze_db}.sales_bronze")
products = spark.table(f"{bronze_db}.products_bronze")
stores   = spark.table(f"{bronze_db}.stores_bronze")
exchange = spark.table(f"{bronze_db}.exchange_rates_bronze")

## 3. Inspeção Inicial

Verificar esquema e primeiros registros para garantir consistência.


In [0]:
# Conferir esquemas e amostras
sales.printSchema()
products.printSchema()
stores.printSchema()
exchange.printSchema()

root
 |-- order_number: integer (nullable = true)
 |-- line_item: integer (nullable = true)
 |-- order_date: date (nullable = true)
 |-- delivery_date: date (nullable = true)
 |-- customerkey: integer (nullable = true)
 |-- storekey: integer (nullable = true)
 |-- productkey: integer (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- currency_code: string (nullable = true)

root
 |-- productkey: integer (nullable = true)
 |-- product_name: string (nullable = true)
 |-- brand: string (nullable = true)
 |-- color: string (nullable = true)
 |-- unit_cost_usd: string (nullable = true)
 |-- unit_price_usd: string (nullable = true)
 |-- subcategorykey: integer (nullable = true)
 |-- subcategory: string (nullable = true)
 |-- categorykey: integer (nullable = true)
 |-- category: string (nullable = true)

root
 |-- storekey: integer (nullable = true)
 |-- country: string (nullable = true)
 |-- state: string (nullable = true)
 |-- square_meters: integer (nullable = true)
 |-- open_

## 4. Junções (Joins)

- `sales_bronze` ↔ `products_bronze` por `productkey`  
- resultado ↔ `stores_bronze` por `storekey`  
- para normalização cambial, join com `exchange_bronze` usando `order_date` e `currency_code`  




In [0]:
# Join com produtos
df = (
    sales.alias("s")
         .join(products.alias("p"), col("s.productkey") == col("p.productkey"), "left")
         .select("s.*", "p.category", "p.subcategory", "p.unit_price_usd")
)

# Join com lojas
df = (
    df.join(stores.alias("st"), df["storekey"] == col("st.storekey"), "left")
      .select(df["*"], "st.country", "st.state", "st.square_meters")
)

# Padronizar formatos de data
df = df.withColumn("order_date", to_date("order_date", "yyyy-MM-dd"))
exchange = exchange.withColumn("date", to_date("date", "yyyy-MM-dd"))




## 5. Conversão de Moeda para USD

- Unir com `exchange` para obter taxa de câmbio  
- Remover símbolo de cifrão e converter `unit_price_usd` para double  
- Calcular `total_sales_usd = quantity * unit_price_usd * exchange`  



In [0]:
# Join com exchange usando order_date + currency_code
df = (
    df.join(exchange.alias("e"),
            (df.order_date == col("e.date")) &
            (df.currency_code == col("e.currency")),
            "left")
)

# Limpar unit_price_usd: remover cifrão e vírgula
df = df.withColumn(
    "unit_price_usd",
    regexp_replace("unit_price_usd", "[$,]", "").cast(DoubleType())
)

# Calcular total_sales_usd
df = df.withColumn(
    "total_sales_usd",
    (col("quantity") * col("unit_price_usd") * col("e.exchange")).cast(DoubleType())
)



## 6. Tratamento de Dados

- Preencher valores nulos críticos  
- Ajustar tipos de colunas onde necessário  
- Remover colunas intermediárias  




In [0]:
# Preenchimento de nulos após joins
df = df.fillna({
    "category": "unknown",
    "subcategory": "unknown",
    "exchange": 1.0
})

df = df.withColumn("square_meters", col("square_meters").cast(IntegerType()))

# Remover colunas temporárias
df = df.drop("currency_code", "e.date", "e.currency", "e.exchange")



## 7. Verificação Final

Visualizar esquema e sample de colunas-chave antes da escrita.



In [0]:
df.printSchema()
df.select("order_date", "unit_price_usd", "quantity", "total_sales_usd").show(10, truncate=False)

root
 |-- order_number: integer (nullable = true)
 |-- line_item: integer (nullable = true)
 |-- order_date: date (nullable = true)
 |-- delivery_date: date (nullable = true)
 |-- customerkey: integer (nullable = true)
 |-- storekey: integer (nullable = true)
 |-- productkey: integer (nullable = true)
 |-- quantity: integer (nullable = true)
 |-- category: string (nullable = false)
 |-- subcategory: string (nullable = false)
 |-- unit_price_usd: double (nullable = true)
 |-- country: string (nullable = true)
 |-- state: string (nullable = true)
 |-- square_meters: integer (nullable = true)
 |-- date: date (nullable = true)
 |-- currency: string (nullable = true)
 |-- exchange: double (nullable = false)
 |-- total_sales_usd: double (nullable = true)

+----------+--------------+--------+------------------+
|order_date|unit_price_usd|quantity|total_sales_usd   |
+----------+--------------+--------+------------------+
|2016-01-01|68.0          |1       |94.41120000000001 |
|2016-01-01|427.

## 8. Escrita da Camada Silver

Gravar como Delta Table e registrar no catálogo `silver`.



In [0]:
# Gravar no DBFS em formato Delta
df.write \
  .format("delta") \
  .mode("overwrite") \
  .save(silver_path)

# Criar database e tabela no catálogo SQL
spark.sql(f"CREATE DATABASE IF NOT EXISTS {silver_db}")
spark.sql(f"""
  CREATE TABLE IF NOT EXISTS {silver_db}.sales_silver
  USING DELTA
  LOCATION '{silver_path}'
""")


Out[33]: DataFrame[]

## 9. Próximos Passos

- Realizar agregações temporais na camada Gold
- Calcular métricas como `total_sales_usd` por mês e por loja/subcategoria  
- Preparar séries temporais contínuas para modelagem preditiva (SARIMA, Prophet)
